# PreProcess

In [72]:
#Read DataSet and Stop words

with open ("hafez.txt", "r") as dataset:
    data = dataset.read().splitlines()

with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stop_words = fa_stop_words.read().splitlines()    

In [73]:
print("data:", "\n1: ", data[0], "\n2: ", data[1], "\n3: ", data[2])
print("\nstop_words:", "\n1: ", stop_words[18], "\n2: ", stop_words[19], "\n3: ", stop_words[20])

data: 
1:  ﻿ 
2:  الا يا ايها الساقي ادر كاسا و ناولها 
3:  كه عشق آسان نمود اول ولي افتاد مشكل‌ها

stop_words: 
1:  آباد 
2:  آخ 
3:  آخر




---



In [ ]:
!pip install hazm
from hazm import *

In [84]:
normalizer = Normalizer()
normalized_data = []
tokenized_data = []
clear_data = []
clear_sentences = []


for line in data:                              #### Normalize
  if line and line != '\ufeff':
    normalized_data.append(normalizer.normalize(line))


for line in normalized_data:                   #### Tokenize
  tokenized_data.append(word_tokenize(line))


for line in tokenized_data:                    #### Remove stop words
  clear_line = []
  for word in line:
    if word not in stop_words:
      clear_line.append(word)
  clear_data.append(clear_line)



for line in clear_data:                        #### Convert words to sentences
  clear_sentences.append(" ".join(line))

clear_sentences = [item for item in clear_sentences if item]



print("Normalized: ", normalized_data[0])
print("Tokenized: ", tokenized_data[0])
print("Cleared: ", clear_data[0])
print("Sentence: ", clear_sentences[0])

Normalized:  الا یا ایها الساقی ادر کاسا و ناولها
Tokenized:  ['الا', 'یا', 'ایها', 'الساقی', 'ادر', 'کاسا', 'و', 'ناولها']
Cleared:  ['ایها', 'الساقی', 'ادر', 'کاسا', 'ناولها']
Sentence:  ایها الساقی ادر کاسا ناولها
